# Convertendo a base de dados de CSV para Parquet

- **Objetivo**: Converter um arquivo CSV para o formato Parquet, que é mais eficiente para armazenamento e consulta de grandes volumes de dados.

- **Motivação**: Atualmente é praticamente impossível trabalhar com a base de dados bruta em CSV que tem mais de 20GB.

In [ ]:
import pandas as pd

# Carrega os dados do CSV (pode demorar alguns minutos)
raw_data = pd.read_csv("DB/dependencies-1.6.0-2020-01-12.csv", dtype={9: str})

In [ ]:
# Salva os dados em Parquet (novamente, pode demorar alguns minutos)
raw_data.to_parquet("DB/dependencies-1.6.0-2020-01-12.parquet")

In [ ]:
pd.set_option("display.max_rows", 50)
display(raw_data)

## Filtrando a base de dados
- **Objetivo**: Filtrar a base de dados para manter apenas as versões mais recentes de cada dependência.

- **Motivação**: A base de dados contém muitas versões antigas de dependências, o que torna a análise mais complexa e menos eficiente. Manter apenas as versões mais recentes facilita a análise e reduz o tamanho do arquivo.

In [ ]:
import pandas as pd
from packaging import version
import numpy as np

# Carrega o arquivo Parquet
df = pd.read_parquet("DB/dependencies-1.6.0-2020-01-12.parquet")

In [ ]:
# ESSA OPERAÇÃO DEMOROU 41 MINUTOS COM A SEGUINTE CONFIGURAÇÃO:
# CPU Intel(R) Xeon(R) CPU E5-2680 v3 @ 2.50GHz
# RAM 256 GB

def parse_version_safe(version_str):
    """Converte string de versão para objeto Version do packaging"""
    try:
        # Remove prefixos comuns e limpa a string
        clean_version = str(version_str).strip().lstrip('v')
        # Remove sufixos como '+build' ou '-alpha'
        clean_version = clean_version.split('+')[0].split('-')[0]
        return version.Version(clean_version)
    except:
        # Para versões inválidas, retorna uma versão muito baixa
        return version.Version("0.0.0")

# Aplica parsing de versão de forma vetorizada
print("Parsing versions...")
df['version_obj'] = df['Version Number'].apply(parse_version_safe)

# Agrupa por Project Name E Platform para encontrar versão máxima
# (importante porque o mesmo projeto pode existir em plataformas diferentes)
print("Finding latest versions...")
latest_versions = df.groupby(['Project Name', 'Platform'])['version_obj'].transform('max')

# Filtra mantendo apenas as versões mais recentes
print("Filtering dataframe...")
df_latest = df[df['version_obj'] == latest_versions].copy()

# Remove coluna auxiliar para economizar memória
df_latest.drop('version_obj', axis=1, inplace=True)

print(f"DataFrame original: {len(df):,} linhas")
print(f"DataFrame filtrado: {len(df_latest):,} linhas")
print(f"Redução: {((len(df) - len(df_latest)) / len(df) * 100):.1f}%")

# Verifica o resultado para o projeto git
print("\nExemplo - versões do git restantes:")
git_versions = df_latest[df_latest["Project Name"] == "git"]["Version Number"].unique()
print(f"Git versions: {sorted(git_versions)}")

In [ ]:
# Salva o DataFrame filtrado em Parquet
df_latest.to_parquet("DB/dependencies-1.6.0-latest_versions.parquet", index=False)